In [1]:
import json
import pickle

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from indra_db.util.content_scripts import get_text_content_from_pmids
from indra_db.util.content_scripts import get_stmts_with_agent_text_like
from indra_db.util.content_scripts import get_text_content_from_stmt_ids

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator

from adeft_indra.s3 import model_to_s3
from adeft_indra.ground import gilda_ground

In [113]:
shortforms = ['RSE']
genes = []
families = {}
groundings = [f'HGNC:{get_hgnc_id(gene)}' for gene in genes]
for family, members in families.items():
    genes.extend(members)
    groundings.append(f'FPLX:{family}')
with open('../data/entrez_all_pmids.json', 'r') as f:
    all_pmids = json.load(f)

In [114]:
entrez_texts = []
entrez_refs = set()
for gene, grounding in zip(genes, groundings):
    try:
        pmids = all_pmids[gene]
    except KeyError:
        continue
    _, content = get_text_content_from_pmids(pmids)
    entrez_texts.extend([(universal_extract_text(text), grounding)
                          for text in content.values() if text])
    entrez_refs.update(content.keys())

In [115]:
miners = dict()
all_texts = set()
for shortform in shortforms:
    stmts = get_stmts_with_agent_text_like(shortform)[shortform]
    _, content = get_text_content_from_stmt_ids(stmts)
    shortform_texts = [universal_extract_text(text, contains=shortforms)
                       for ref, text in content.items() if text and ref not in entrez_refs]
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(shortform_texts)
    all_texts |= set(shortform_texts)

It's then necessary to check if Acromine produced the correct results. We must fix errors manually

In [116]:
top = miners['RSE'].top()

In [48]:
top

[('refractory status epilepticus', 39.13333333333334),
 ('extract', 9.538461538461538),
 ('repeated sprint exercise', 5.0),
 ('element', 4.923076923076923),
 ('stability element', 4.8),
 ('rna stability element', 3.6666666666666665),
 ('standardized ginseng extract', 3.0),
 ('refractory se', 2.5),
 ('the rsv stability element', 2.5),
 ('treat refractory status epilepticus', 2.5),
 ('junction squamous epithelial', 2.0),
 ('in refractory status epilepticus', 2.0),
 ('relationship and sexuality education', 2.0),
 ('present of multiple stimuli can actually facilitate performance the redundant signal effect',
  2),
 ('udy criteria of intensive care unit icu general anaesthesia treat refractory status epilepticus',
  2),
 ('ginseng extract', 1.6),
 ('squamous epithelial', 1.6),
 ('root', 1.5),
 ('raspberry extract', 1.3333333333333333),
 ('sachalinensis root', 1.3333333333333333),
 ('radix scutellariae extract', 1.3333333333333333),
 ('rsv rna stability element', 1.3333333333333333),
 ('trea

In [75]:
longforms0 = miners['RSE'].get_longforms()

In [76]:
list(enumerate(longforms0))

[(0, ('refractory status epilepticus', 39.13333333333334)),
 (1, ('extract', 9.538461538461538)),
 (2, ('repeated sprint exercise', 5.0)),
 (3, ('element', 4.923076923076923)),
 (4, ('refractory se', 2.5)),
 (5, ('junction squamous epithelial', 2.0)),
 (6, ('relationship and sexuality education', 2.0)),
 (7,
  ('present of multiple stimuli can actually facilitate performance the redundant signal effect',
   2)),
 (8, ('root', 1.5))]

In [77]:
longforms0 = [(longform, score) for i, (longform, score) in enumerate(longforms0)
              if i not in [1, 3, 7, 8]]

In [78]:
list(enumerate(top))

[(0, ('refractory status epilepticus', 39.13333333333334)),
 (1, ('extract', 9.538461538461538)),
 (2, ('repeated sprint exercise', 5.0)),
 (3, ('element', 4.923076923076923)),
 (4, ('stability element', 4.8)),
 (5, ('rna stability element', 3.6666666666666665)),
 (6, ('standardized ginseng extract', 3.0)),
 (7, ('refractory se', 2.5)),
 (8, ('the rsv stability element', 2.5)),
 (9, ('treat refractory status epilepticus', 2.5)),
 (10, ('junction squamous epithelial', 2.0)),
 (11, ('in refractory status epilepticus', 2.0)),
 (12, ('relationship and sexuality education', 2.0)),
 (13,
  ('present of multiple stimuli can actually facilitate performance the redundant signal effect',
   2)),
 (14,
  ('udy criteria of intensive care unit icu general anaesthesia treat refractory status epilepticus',
   2)),
 (15, ('ginseng extract', 1.6)),
 (16, ('squamous epithelial', 1.6)),
 (17, ('root', 1.5)),
 (18, ('raspberry extract', 1.3333333333333333)),
 (19, ('sachalinensis root', 1.3333333333333333

In [79]:
longforms0.extend((longform, score) for i, (longform, score) in enumerate(top)
                 if i in [5, 6])

In [80]:
longforms0.append(('rsv stability element', 2.5))

In [81]:
longforms = longforms0

In [82]:
longforms.sort(key=lambda x: -x[1])

In [83]:
longforms, scores = zip(*longforms)

In [84]:
longforms

('refractory status epilepticus',
 'repeated sprint exercise',
 'rna stability element',
 'standardized ginseng extract',
 'refractory se',
 'rsv stability element',
 'junction squamous epithelial',
 'relationship and sexuality education')

In [85]:
grounding_map = {}
names = {}
for longform in longforms:
    grounding = gilda_ground(longform)
    if grounding[0]:
        grounding_map[longform] = f'{grounding[0]}:{grounding[1]}'
        names[grounding_map[longform]] = grounding[2]

In [86]:
grounding_map

{}

In [87]:
names

{}

In [88]:
grounding_map, names, pos_labels = ground_with_gui(longforms, scores, grounding_map=grounding_map, names=names)

In [89]:
result = (grounding_map, names, pos_labels)

In [90]:
result

({'junction squamous epithelial': 'ungrounded',
  'refractory se': 'ungrounded',
  'refractory status epilepticus': 'MESH:D013226',
  'relationship and sexuality education': 'ungrounded',
  'repeated sprint exercise': 'MESH:D000076962',
  'rna stability element': 'ungrounded',
  'rsv stability element': 'ungrounded',
  'standardized ginseng extract': 'ungrounded'},
 {'MESH:D013226': 'Status Epilepticus',
  'MESH:D000076962': 'High-Intensity Interval Training'},
 ['MESH:D000076962', 'MESH:D013226'])

In [117]:
grounding_map, names, pos_labels = ({'junction squamous epithelial': 'ungrounded',
  'refractory se': 'ungrounded',
  'refractory status epilepticus': 'MESH:D013226',
  'relationship and sexuality education': 'ungrounded',
  'repeated sprint exercise': 'MESH:D000076962',
  'rna stability element': 'ungrounded',
  'rsv stability element': 'ungrounded',
  'standardized ginseng extract': 'ungrounded'},
 {'MESH:D013226': 'Status Epilepticus',
  'MESH:D000076962': 'High-Intensity Interval Training'},
 ['MESH:D000076962', 'MESH:D013226'])

In [118]:
grounding_dict = {'RSE': grounding_map}

In [119]:
classifier = AdeftClassifier('RSE', pos_labels=pos_labels)

In [120]:
param_grid = {'C': [100.0], 'max_features': [10000]}

In [121]:
labeler = AdeftLabeler(grounding_dict)

In [122]:
corpus = labeler.build_from_texts(shortform_texts)

In [123]:
corpus.extend(entrez_texts)

In [124]:
texts, labels = zip(*corpus)

In [125]:
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=8)

INFO: [2020-01-27 15:54:26] /Users/albertsteppi/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
/Users/albertsteppi/.virtualenvs/py37/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
INFO: [2020-01-27 15:54:50] /Users/albertsteppi/adeft/adeft/modeling/classify.py - Best f1 score of 0.8223809523809523 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [126]:
classifier.stats

{'label_distribution': {'MESH:D013226': 32,
  'ungrounded': 15,
  'MESH:D000076962': 3},
 'f1': {'mean': 0.8223809523809523, 'std': 0.07956229465116456},
 'precision': {'mean': 0.7892857142857144, 'std': 0.06428571428571428},
 'recall': {'mean': 0.8642857142857142, 'std': 0.1203735681882337}}

In [127]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [128]:
d = disamb.disambiguate(shortform_texts)

In [130]:
a = [text for pred, text in zip(d, shortform_texts)if pred[0] == 'MESH:D013226']

In [131]:
a[7]

'Refractory status epilepticus (RSE) is an emergency with high mortality requiring neurointensive care. Treatment paradigms include first-generation antiepileptic drugs (AEDs) and anesthetics. Lacosamide (LCM) is a new AED, holding promise as a potent treatment option for RSE. High-level evidence regarding safety and efficacy in the treatment of RSE is lacking. The objective of the study was to evaluate the safety profile and efficacy of intravenous (i.v.) LCM as an add-on treatment in adult RSE patients. All consecutive RSE patients treated in the intensive care units (ICUs) of an academic tertiary care center between 2005 and 2011 were included. Severity of status epilepticus (SE) was graded by the SE Severity Scale (STESS), and SE etiology was categorized according to the guidelines of the International League Against Epilepsy (ILAE). Outcomes were seizure control, RSE duration, and death. Of 111 RSE patients, 53 % were treated with LCM. Twenty-five patients with hypoxic-ischemic en

In [132]:
disamb.dump('RSE', '../results')

In [68]:
from adeft.disambiguate import load_disambiguator, load_disambiguator_directly

In [42]:
disamb.classifier.training_set_digest

'baeb1ba13324f4192620577a2d3e6ab9'

In [133]:
model_to_s3(disamb)

In [76]:
d.disambiguate(texts[0])

('CHEBI:CHEBI:29016',
 'arginine',
 {'CHEBI:CHEBI:29016': 0.9667882794828576,
  'CHEBI:CHEBI:79': 0.004343534773612251,
  'HGNC:651': 0.00798509086185913,
  'HGNC:77': 0.00973784879530277,
  'HGNC:9965': 0.011145246086368127})

In [79]:
print(d.info())

Disambiguation model for ARG

Produces the disambiguations:
	(-)-Arctigenin*	CHEBI:CHEBI:79
	ABL2*	HGNC:77
	AREG*	HGNC:651
	RERE*	HGNC:9965
	arginine*	CHEBI:CHEBI:29016

Training data had class balance:
	AREG*	157
	ABL2*	88
	RERE*	39
	arginine*	15
	(-)-Arctigenin*	2

Classification Metrics:
	F1 score:	0.81188
	Precision:	0.82394
	Recall:		0.8205

* Positive labels
See Docstring for explanation



In [32]:
a = load_disambiguator('AR')

In [33]:
a.disambiguate('Androgen')

('HGNC:644',
 'AR',
 {'FPLX:ADRB': 4.719192246728643e-10,
  'GO:GO:0007340': 1.3624879544267196e-09,
  'HGNC:381': 5.2828214761836554e-12,
  'HGNC:644': 0.9999384718638125,
  'HGNC:651': 1.8656671170325e-12,
  'ungrounded': 6.152629463170691e-05})

In [36]:
logit = d.classifier.estimator.named_steps['logit']

In [37]:
logit.classes_

array(['FPLX:AQP', 'FPLX:Macrophage_inflammatory_pro', 'MESH:D000072277',
       'polymer', 'ungrounded'], dtype='<U32')

In [41]:
model_to_s3(disamb)

In [44]:
classifier.feature_importances()['FPLX:RAC']

[('rac1', 3.0203950681340763),
 ('cdc42', 0.46463658443237615),
 ('cells', 0.26067826443238284),
 ('rhoa', 0.2284452268653803),
 ('rho', 0.201414697794678),
 ('rac1b', 0.1862119230497441),
 ('actin', 0.16300556582714468),
 ('gtpases', 0.15543881285531247),
 ('rictor', 0.14896197766149943),
 ('pak1', 0.14776120596579687),
 ('jnk', 0.13802477317328446),
 ('activation', 0.13739172186857945),
 ('s1p', 0.12195584486540219),
 ('thrombin', 0.12054088946040988),
 ('endothelial', 0.11948523924516169),
 ('proteins', 0.11834598588132215),
 ('cell', 0.11418065253318789),
 ('phox', 0.11144187989246301),
 ('chimaerin', 0.1102266733731222),
 ('elmo1', 0.10886660370678143),
 ('vav', 0.10685654194092864),
 ('ros', 0.1017412296369346),
 ('gtp', 0.09970566367485924),
 ('escc', 0.09632367768212988),
 ('ras', 0.09346518511044424),
 ('nef', 0.08938792674953275),
 ('ubiquitylation', 0.0893305800391189),
 ('p85', 0.08885763883676143),
 ('gtpase', 0.08833886568389247),
 ('figure', 0.087475101884638),
 ('comple

In [46]:
d = load_disambiguator('ALK', '../results')

In [47]:
d.info()

'Disambiguation model for ALK\n\nProduces the disambiguations:\n\tALK*\tHGNC:427\n\tAlkaline Phosphatase*\tMESH:D000469\n\tRSTK1*\tFPLX:RSTK1\n\nTraining data had class balance:\n\tALK*\t1181\n\tRSTK1*\t155\n\tAlkaline Phosphatase*\t6\n\nClassification Metrics:\n\tF1 score:\t0.97493\n\tPrecision:\t0.97367\n\tRecall:\t\t0.97767\n\n* Positive labels\nSee Docstring for explanation\n'

In [48]:
print(d.info())

Disambiguation model for ALK

Produces the disambiguations:
	ALK*	HGNC:427
	Alkaline Phosphatase*	MESH:D000469
	RSTK1*	FPLX:RSTK1

Training data had class balance:
	ALK*	1181
	RSTK1*	155
	Alkaline Phosphatase*	6

Classification Metrics:
	F1 score:	0.97493
	Precision:	0.97367
	Recall:		0.97767

* Positive labels
See Docstring for explanation



In [49]:
model_to_s3(d)

In [58]:
d = load_disambiguator('TAK', '../results')

In [59]:
print(d.info())

Disambiguation model for TAK

Produces the disambiguations:
	MAP3K7*	HGNC:6859
	Takayasu Arteritis*	MESH:D013625

Training data had class balance:
	MAP3K7*	327
	Ungrounded*	241
	Ungrounded	36
	Takayasu Arteritis*	13

Classification Metrics:
	F1 score:	0.94043
	Precision:	0.93685
	Recall:		0.94658

* Positive labels
See Docstring for explanation



In [57]:
model_to_s3(d)

In [60]:
from adeft import available_shortforms

In [63]:
print(d.info())

Disambiguation model for TAK

Produces the disambiguations:
	MAP3K7*	HGNC:6859
	Takayasu Arteritis*	MESH:D013625

Training data had class balance:
	MAP3K7*	327
	Ungrounded*	241
	Ungrounded	36
	Takayasu Arteritis*	13

Classification Metrics:
	F1 score:	0.94043
	Precision:	0.93685
	Recall:		0.94658

* Positive labels
See Docstring for explanation



In [64]:
d.classifier.feature_importances()

TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

In [2]:
from adeft import __version__

In [3]:
__version__

'0.5.5'

In [4]:
from adeft.disambiguate import load_disambiguator_directly

In [59]:
d = load_disambiguator_directly('../results/TEK/')

In [60]:
print(d.info())

Disambiguation model for TEK

Produces the disambiguations:
	TEK*	HGNC:11724

Training data had class balance:
	TEK*	217
	Ungrounded	9

Classification Metrics:
	F1 score:	0.99316
	Precision:	0.98646
	Recall:		1.0

* Positive labels
See Docstring for explanation



In [61]:
model_to_s3(d)

In [32]:
d.grounding_dict

{'TEC': {'ternary elongation complex': 'transcription elongation complex',
  'thermal expansion coefficient': 'ungrounded',
  'thromboembolic complications': 'ungrounded',
  'thymic epithelial cells': 'NCIT:C33771',
  'thymic epithelial cells line': 'NCIT:C33771',
  'thyrocyte': 'MESH:D000072637',
  'thyroid epithelial cells': 'MESH:D000072637',
  'tissue engineered construct': 'ungrounded',
  'total electron content': 'ungrounded',
  'tracheal epithelial cells': 'ungrounded',
  'transcription elongation complex': 'transcription elongation complex',
  'triethyl citrate': 'PUBCHEM:6506',
  'tubular epithelial cells': 'NCIT:C61147',
  'tumor associated endothelial cells': 'NCIT:C37088',
  'tumor endothelial cells': 'NCIT:C37088'},
 'TECs': {'thymic epithelial cells': 'NCIT:C33771',
  'thymus epithelial cells': 'NCIT:C33771',
  'tissue engineered constructs': 'ungrounded',
  'transcription elongation complexes': 'transcription elongation complex',
  'tubular epithelial cells': 'NCIT:C6114

In [25]:
!python -m adeft.download --update

100% [......................................................] 1181008 / 1181008

In [26]:
from adeft import available_shortforms

In [27]:
len(available_shortforms)

72

In [28]:
available_shortforms

{'PC': 'PC',
 'DOG1': 'DOG1',
 'TEK': 'TEK',
 'EMT': 'EMT',
 'SP': 'SP',
 'PCS': 'PCS',
 'CKI': 'CKI',
 'PE': 'PE',
 'RET': 'RET',
 'TEC': 'TEC',
 'ROS': 'ROS',
 'NP': 'NP:NP_S',
 'NPs': 'NP:NP_S',
 'MS': 'MS',
 'MT': 'MT',
 'BP': 'BP',
 'GH': 'GH',
 'AD': 'AD',
 'GT': 'GT',
 'SNS': 'SNS',
 'GARP': 'GARP',
 'DA': 'DA',
 'BCR': 'BCR',
 'GR': 'GR',
 'HIR': 'HIR',
 'IR': 'IR',
 'HK2': 'HK2',
 'ARF': 'ARF',
 'CS': 'CS',
 'UFO': 'UFO',
 'LAK': 'LAK',
 'EC': 'EC',
 'ARG': 'ARG',
 'MOS': 'MOS',
 'STD': 'STD',
 'PD1': 'PD1',
 'TGH': 'TGH',
 'PKD': 'PKD',
 'MIP': 'MIP',
 'RA': 'RA',
 'PCP': 'PCP',
 'EAG': 'EAG',
 'PI': 'PI',
 'PS': 'PS',
 'PA': 'PA',
 'MB': 'MB',
 'HA': 'HA',
 'FES': 'FES',
 'AR': 'AR',
 'HR': 'HR',
 'NE': 'NE',
 'UBC': 'UBC',
 'GSC': 'GSC',
 'AA': 'AA',
 'FER': 'FER',
 'NIS': 'NIS',
 'GC': 'GC',
 'DLK': 'DLK',
 'HK1': 'HK1',
 'CM': 'CM',
 'RB': 'RB:R_B',
 'Rb': 'RB:R_B',
 'LH': 'LH',
 'ER': 'ER',
 'TF': 'TF',
 'PGP': 'PGP',
 'GCA': 'GCA',
 'RK': 'RK',
 'RAC': 'RAC',
 'MCT': 'M

In [33]:
'TEC' in available_shortforms

True

In [34]:
'TECs' in available_shortforms

False

In [35]:
!python -m adeft.download --update

Shortform TEC has multiple adeft modelsThis may lead to unexpected behavior
Shortform TEC has multiple adeft modelsThis may lead to unexpected behavior
100% [........................................................] 194559 / 194559Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/albertsteppi/adeft/adeft/download/__main__.py", line 18, in <module>
    download_models(update=args.update)
  File "/Users/albertsteppi/adeft/adeft/download/download.py", line 65, in download_models
    out=resource_path)
  File "/Users/albertsteppi/.virtualenvs/py37/lib/python3.7/site-packages/wget.py", line 526, in download
    (tmpfile, headers) = ulib.urlretrieve(binurl, tmpfile, callback)
  

In [36]:
!python -m adeft.download --update

100% [......................................................] 1181008 / 1181008